### Spill
  
Å lage spill er en fin måte å øve seg på objektorientert programmering. For å lage spill kommer vi til å bruke `pygame`-pakken. 
    
Du kan finne dokumentasjonen til Pygame på [denne nettsiden](https://www.pygame.org/docs/).

#### Boilerplate
  
Her er et enkelt eksempel som viser hvordan vi kan sette opp et spill med `pygame`.
 
```
import pygame

pygame.init()
screen = pygame.display.set_mode((500, 500))
clock = pygame.time.Clock()

# Main loop
running = True
while running:
    # Event-handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    # Update
    clock.tick(60)

    # Render
    screen.fill("white")
    pygame.display.flip()

pygame.quit()
```

Før hovedløkken initialiserer vi noen objekter. Hovedløkken kan tenkes på som tre deler som gjøres etter hverandre.

1. **Event-handling** skal håndtere input fra brukeren.
2. **Update** skal oppdatere spillobjektene våre.
3. **Render** skal tegne spillobjektene på skjermen.

Kjører vi programmet får vi bare en hvit skjerm. La oss lage et spill ut av dette.

#### Et enkelt spill

La oss lage et enkelt spill som viser hvordan noen grunnleggende elementer i `pygame`.

```
import pygame, random

pygame.init()
screen = pygame.display.set_mode((500, 500))
clock = pygame.time.Clock()

class SpillObjekt:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = (100, 100, 100)
        self.rect = pygame.Rect(x - width // 2, y - height // 2, width, height)

    def update(self):
        self.rect = pygame.Rect(self.x - self.width // 2, self.y - self.height // 2, self.width, self.height)

    def render(self):
        pygame.draw.rect(screen, self.color, self.rect)

class Spiller(SpillObjekt):
    def __init__(self, x, y):
        super().__init__(x, y, width = 10, height = 10)
        self.color = (0, 200, 0)

    def update(self):
        if actions["left"]:
            self.x -= 5
        elif actions["right"]:
            self.x += 5
        if actions["up"]:
            self.y -= 5
        elif actions["down"]:
            self.y += 5

        for fiende in fiender:
            if pygame.Rect.colliderect(self.rect, fiende.rect):
                pygame.quit()
        
        self.rect = pygame.Rect(self.x - self.width // 2, self.y - self.height // 2, self.width, self.height)

class Fiende(SpillObjekt):
    def __init__(self, x, y):
        super().__init__(x, y, width = 20, height = 20)
        self.color = (200, 0, 0)
        self.speed = 5
        self.vx = random.choice([-1, 1])
        self.vy = random.choice([-1, 1])

    def update(self):
        self.x += self.vx * self.speed
        self.y += self.vy * self.speed

        if self.x > screen.get_width() or self.x < 0:
            self.vx *= -1
        if self.y > screen.get_height() or self.y < 0:
            self.vy *= -1

        self.rect = pygame.Rect(self.x - self.width // 2, self.y - self.height // 2, self.width, self.height)

# Initialisere actions-ordbok
actions = {"left" : False, "right" : False, "up" : False, "down" : False}

# Initialisere spillobjekter
spiller = Spiller(screen.get_width() // 2, screen.get_height() // 2)
fiender = [Fiende(random.randint(0, screen.get_width()), random.randint(0, screen.get_height())) for n in range(3)]

running = True
while running:
    # Event-handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                actions["left"] = True
            if event.key == pygame.K_RIGHT:
                actions["right"] = True
            if event.key == pygame.K_UP:
                actions["up"] = True
            if event.key == pygame.K_DOWN:
                actions["down"] = True

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                actions["left"] = False
            if event.key == pygame.K_RIGHT:
                actions["right"] = False
            if event.key == pygame.K_UP:
                actions["up"] = False
            if event.key == pygame.K_DOWN:
                actions["down"] = False
    # Update
    spiller.update()
    for fiende in fiender:
        fiende.update()
    clock.tick(60)

    # Render
    screen.fill("white")
    for fiende in fiender:
        fiende.render()
    spiller.render()
    pygame.display.flip()
```

Phew! La oss se nærmere på noen deler.
- `actions` er en ordbok som holder styr over hvilke knapper som er trykket på.
- `SpillObjekt` er en generell klasse for objekter i spillet vårt. `Spiller` og `Fiende` arver av dette.
- Hvert `SpillObjekt` har en `Rect` (fra `pygame`) som må oppdateres og tegnes på skjermen.
- `Spiller` kan beveges på ved å bruke tastene.
- `Fiende` starter med fart i en tilfeldig diagonal retning og spretter av veggene.
- Når en fiende kolliderer med en spiller, så stoppes spillet.

Man kan utvide og forbedre dette spillet på mange måter! Knock yourself out!